In [ ]:
%matplotlib inline

import os
os.environ['R_User'] = '/usr/local/lib/python3.6/site-packages'

import csv
import pandas
import numpy
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri    
from rpy2.robjects.packages import importr
import glob

import plotly.plotly
# print(plotly.__version__)  
plotly.offline.init_notebook_mode() # run at the start of every notebook
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import peakutils

In [ ]:
def lowestpoint(series):
    return numpy.argmax( numpy.maximum.accumulate(series) - series)

In [ ]:
base = importr('base')

In [ ]:
directory = '/Users/minhdoan/Desktop/AV_ori_original/'

In [ ]:
files = glob.glob(os.path.join(directory,"*"))

In [ ]:
# files = ["/Users/minhdoan/Desktop/AV_ori_original/1_67169001_67206250.RData"]

In [ ]:
eachfile = []
for file in files:
    #print(os.path.basename(file))
    base.load(file)
    array = numpy.array(base.mget(base.ls()) )
    
    a = array[0][0:7,:]
    series_peaks = numpy.amax(a,axis = 0)
    
    cb_peak = numpy.array(series_peaks)
    indices_peaks = peakutils.indexes(cb_peak, thres=0.60, min_dist=700)
    
    
    series_valleys = -numpy.amax(a,axis = 0)
    cb_valley = numpy.array(series_valleys)
    indices_valleys = peakutils.indexes(cb_valley, thres=0.70, min_dist=700)
    
    starts = []
    ends = []   
    
    if len(indices_valleys) > 0:       
        
        # Find the closest foot of the hill: this is index in the valleys series, NOT index of the peaks series
        idx = [(numpy.abs(indices_valleys - peak)).argmin() for peak in indices_peaks]
          
        for i, index in enumerate(idx):

            #--- Find the start ---#           
            if indices_valleys[index] > indices_peaks[i]:
                if index - 1 >= 0:
                    starts.append( indices_valleys[index-1] )
                else:
                    starts.append( lowestpoint(series_peaks[:indices_peaks[i]]) )
            else:
                starts.append( indices_valleys[index] )

            #--- Find the end ---# 
            if indices_valleys[index] > indices_peaks[i]:
                ends.append( indices_valleys[index] )
            else:               
                try: # when the next valley is available
                    ends.append(indices_valleys[index + 1])                                
                except: # when already used up the last valley
                    ends.append( indices_peaks[i] + lowestpoint(series_peaks[indices_peaks[i]:]) )                    
                
                
    else:         
        for i in indices_peaks:
            
            starts.append( lowestpoint(series_peaks[:i]) )
            ends.append( lowestpoint(series_peaks[i:]) )

            
    peak_start_end = []  
    for i,j in enumerate(indices_peaks):
        peak_start_end.append([j, starts[i], ends[i]])
    
    eachfile.append([os.path.basename(file),peak_start_end])
    del(peak_start_end,starts,ends,indices_peaks,indices_valleys,cb_peak,cb_valley,series_peaks,series_valleys,array,a)

In [ ]:
with open("filenames_peak_start_end.csv","w",newline="") as f:  # open("output.csv","wb") for Python 2
    cw = csv.writer(f)
    cw.writerows(r+[""] for r in eachfile)

# Visualization

In [ ]:
trace1 = go.Scatter(
    x=[j for j in range(array[0].shape[1]) ],
    y=series_peaks,
    mode='lines',
    name='Original Plot'
)

trace2 = go.Scatter(
    x=indices_peaks,
    y=[series_peaks[j] for j in indices_peaks],
    mode='markers',
    marker=dict(
        size=8,
        color='rgb(255,0,0)',
        symbol='cross'
    ),
    name='Detected Peaks'
)

# data = [trace1, trace2]
# plotly.offline.iplot(data, filename='plot-with-peaks')

In [ ]:
trace3 = go.Scatter(
    x=[j for j in range(array[0].shape[1]) ],
    y=series_valleys,
    mode='lines',
    name='Original Plot'
)

trace4 = go.Scatter(
    x=indices_valleys,
    y=[series_valleys[j] for j in indices_valleys],
    mode='markers',
    marker=dict(
        size=8,
        color='rgb(255,0,0)',
        symbol='cross'
    ),
    name='Detected Valleys'
)

# data = [trace3, trace4]
# plotly.offline.iplot(data, filename='plot-with-valleys')

In [ ]:
trace5 = go.Scatter(
    x=starts,
    y=[series_peaks[j] for j in starts],
    mode='markers',
    marker=dict(
        size=8,
        color='rgb(255,0,255)',
        symbol='cross'
    ),
    name='Detected Starts'
)

trace6 = go.Scatter(
    x=ends,
    y=[series_peaks[j] for j in ends],
    mode='markers',
    marker=dict(
        size=8,
        color='rgb(0,255,255)',
        symbol='cross'
    ),
    name='Detected Ends'
)


data = [trace1, trace2, trace5, trace6]
plotly.offline.iplot(data, filename='plot-with-peaks_ends')

In [ ]:
eachfile